In [1]:
import os
import pandas as pd
import numpy as np
import tensorflow as tf
import torch.nn.functional as F

2023-03-07 01:45:31.251819: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-03-07 01:45:31.416810: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-03-07 01:45:31.419624: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: :/home/ikonkobo/anaconda3/lib/:/home/ikonkobo/anaconda3/lib/:/home/ikonkobo/anacond

In [2]:
os.chdir("../")
from src.data.request_data import getDataTF, getDataHF
from src.data.processing_data import StackedFormat
from src.models.pipelines import Pipeline
from src.models.encoders import BERTencoder

In [3]:
df = pd.read_csv(f"./inputs_data/data_dyda_da_train.csv", encoding="utf-8",  sep="|")

In [4]:
len(df["Label"].unique())

4

In [5]:
df[["Dialogue_ID", "Label"]].groupby("Dialogue_ID").count().reset_index() #.Label.value_counts()

,Dialogue_ID,Label
0,1,10
1,2,6
2,3,5
3,4,4
4,5,9
...,...,...
11113,11114,9
11114,11115,6
11115,11116,16
11116,11117,4


In [6]:
dimDialogAct, Contexts, Labels = StackedFormat("dyda_da", 2).get_contexts_labels()

2023-03-07 01:45:34.210662: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: :/home/ikonkobo/anaconda3/lib/:/home/ikonkobo/anaconda3/lib/:/home/ikonkobo/anaconda3/lib/
2023-03-07 01:45:34.210682: W tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:265] failed call to cuInit: UNKNOWN ERROR (303)
2023-03-07 01:45:34.210713: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (ikonkobo): /proc/driver/nvidia/version does not exist
2023-03-07 01:45:34.211003: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, re

In [7]:
dimDialogAct

4

In [8]:
# BERTencoder('bert-base-uncased').embedding(Contexts[2][0])

In [9]:
# Labels[2]

## Testing Pipeline working

In [10]:
perf = Pipeline("dyda_da", 2, 'bert-base-uncased', 2, "MLP").summary_exec()

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.bias', 'cls.seq_relationship.bias', 

2/2 [==============================] - 0s 3ms/step


In [11]:
perf

,dataset_name,encoder_model,decoder_model,performance
0,dyda_da,bert-base-uncased,MLP,0.238095
